In [11]:
import pandas as pd
import pprint as pp
import numpy as np
import streamlit as st
import requests
from requests import post, get
import base64
import urllib
import os
import json
from dotenv import load_dotenv
import plotly.express as px

In [2]:
# Spotify Developer Dashboard details
load_dotenv()
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

# We first request a token to the Spotify Accounts Service, which will be used later on to access the Spotify Web API
# To get our access token, we need to pass our client ID, client Secret and grant_type
def get_token():
    auth_string = client_id + ':' + client_secret
    auth_bytes = auth_string.encode("utf-8")
    # Spotify requests to encode the client ID and client secret using base64
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}
    # Make the request
    result = post(url,headers= headers, data= data)
    # Convert the result (json) into a python dictionary
    json_result = json.loads(result.content) 
    token = json_result["access_token"]
    return token

In [3]:
token= get_token()

In [4]:
# Function to construct the header to send a request
def get_auth_header(token):
    return{"Authorization": "Bearer " + token}

In [5]:
# Get new releases
def get_new_releases(country):
    url= f"https://api.spotify.com/v1/browse/new-releases?country={country}&limit=1"
    headers = get_auth_header(token)
    result = get(url, headers=headers)
    json_result = json.loads(result.content)['albums']['items'][0]['name']
    return json_result

In [6]:
# Get the new release ID, so that later we can search for the tracks
def get_new_releases_id(country):
    url= f"https://api.spotify.com/v1/browse/new-releases?country={country}&limit=1"
    headers = get_auth_header(token)
    result = get(url, headers=headers)
    json_result = json.loads(result.content)['albums']['items'][0]['id']
    return json_result

In [29]:
# With the ID, get the tracks of that album
def tracks_from_album(country):
    album_id= get_new_releases_id(country)
    url= f"https://api.spotify.com/v1/albums/{album_id}/tracks?limit=2"
    headers = get_auth_header(token)
    result = get(url, headers=headers)
    json_result = json.loads(result.content)#['items']#[0]['name']
    return json_result

In [30]:
test= tracks_from_album('BA')
pp.pprint(test)

{'href': 'https://api.spotify.com/v1/albums/5r36AJ6VOJtp00oxSkBZ5h/tracks?offset=0&limit=2',
 'items': [{'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6KImCVD70vtIoJWnq6nGn3'},
                         'href': 'https://api.spotify.com/v1/artists/6KImCVD70vtIoJWnq6nGn3',
                         'id': '6KImCVD70vtIoJWnq6nGn3',
                         'name': 'Harry Styles',
                         'type': 'artist',
                         'uri': 'spotify:artist:6KImCVD70vtIoJWnq6nGn3'}],
            'available_markets': ['AR',
                                  'AU',
                                  'AT',
                                  'BE',
                                  'BO',
                                  'BR',
                                  'BG',
                                  'CA',
                                  'CL',
                                  'CO',
                                  'CR',
                                  'C

In [38]:
for idx, track in enumerate(test):
    name=test['items'][idx]['name']
    number= test['items'][idx]['track_number']
    pp.pprint(number)

1
2


IndexError: list index out of range

In [45]:
#I need to extract all the songs, probably print them in a for loop

# Map

In [2]:
df= pd.read_excel(r'C:\Users\sofia\OneDrive\Documentos\GitHub\bigdataproject\app\data\country-available-final.xlsx')
df

,country,capital,cap_lat,cap_lon,country_code,continent
0,Palestine,Jerusalem,31.766667,35.233333,PS,Asia
1,Nauru,Yaren,-0.547700,166.920867,NR,Australia
2,Albania,Tirana,41.316667,19.816667,AL,Europe
3,Algeria,Algiers,36.750000,3.050000,DZ,Africa
4,Andorra,Andorra la Vella,42.500000,1.516667,AD,Europe
...,...,...,...,...,...,...
178,Vietnam,Hanoi,21.033333,105.850000,VN,Asia
179,Zambia,Lusaka,-15.416667,28.283333,ZM,Africa
180,Zimbabwe,Harare,-17.816667,31.033333,ZW,Africa
181,Hong Kong,NaN,0.000000,0.000000,HK,Asia


In [9]:
df.iloc[115,4]

nan

In [10]:
df.iloc[115,4]='NA'

In [11]:
df.iloc[115,:]

country           Namibia
capital          Windhoek
cap_lat        -22.566667
cap_lon         17.083333
country_code           NA
continent          Africa
Name: 115, dtype: object

In [12]:
df['new_releases']= [get_new_releases(df.loc[index,'country_code']) for index in range(len(df))]

In [13]:
df.head()

,country,capital,cap_lat,cap_lon,country_code,continent,new_releases
0,Palestine,Jerusalem,31.766667,35.233333,PS,Asia,TUYA
1,Nauru,Yaren,-0.547700,166.920867,NR,Australia,BLONDE16
2,Albania,Tirana,41.316667,19.816667,AL,Europe,Ku Je Ti (feat. Ricky Rich & Dafina Zeqiri)
3,Algeria,Algiers,36.750000,3.050000,DZ,Africa,YA LE3DOUWA
4,Andorra,Andorra la Vella,42.500000,1.516667,AD,Europe,"PARANOÏA, ANGELS, TRUE LOVE"


In [16]:
fig = px.scatter_geo(df, lat='cap_lat', lon= 'cap_lon',
                     hover_name='new_releases', color='country'
                     ) #text= 'country'

# Update the layout with the desired colors
fig.update_layout(
    geo=dict(
        bgcolor='black',  # Set the background color to black
        showland=True,
        landcolor='white',  # Set the color of land areas to white
        showocean=True,
        oceancolor='black',  # Set the color of ocean areas to black
        showlakes=True,
        lakecolor='white'  # Set the color of lakes to white
    ),
    font=dict(
        color='white'  # Set the text color to white
    )
)
fig.show()